In [166]:
import sp3
import astropy.time
import pathlib

def download_sp3_file(sp3_id: str, start_date: str, end_date: str, download_dir: pathlib.Path):
    # Ensure the download directory exists
    download_dir.mkdir(parents=True, exist_ok=True)

    # Define the SP3 ID and the time range
    satellite_id = sp3.NoradId(sp3_id)
    start = astropy.time.Time(start_date)
    end = astropy.time.Time(end_date)
    
    # Calculate the range to download SP3 files without interpolation
    delta_days = astropy.time.TimeDelta(1, format='jd')  # 1 day step to ensure all files are covered
    obstime = start + delta_days * range(int((end - start).jd) + 1)  # Cover entire range day by day

    # Download SP3 files
    for single_day in obstime:
        try:
            sp3.itrs(
                id=satellite_id,
                obstime=astropy.time.Time([single_day.iso]),
                download_directory=download_dir,
                window=1,  # Minimal window since we're not interpolating
                degree=1  # Minimal degree since we're not interpolating
            )
            print(f"Downloaded SP3 file for {single_day.iso} in {download_dir}")
        except Exception as e:
            print(f"Failed to download or find SP3 file for {single_day.iso}: {str(e)}")

# Example usage: Adjust the NORAD ID and time range as needed
download_sp3_file("25933", "2022-12-31T00:00:00", "2023-01-02T00:00:00", pathlib.Path("sp3_download"))

import orekit
from orekit.pyhelpers import setup_orekit_curdir, datetime_to_absolutedate
from datetime import datetime, timedelta

# orekit.pyhelpers.download_orekit_data_curdir()
vm = orekit.initVM()
setup_orekit_curdir("misc/orekit-data.zip")

from org.orekit.frames import FramesFactory, ITRFVersion
from org.orekit.utils import PVCoordinates
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.utils import PVCoordinates, IERSConventions
from org.orekit.orbits import KeplerianOrbit, CartesianOrbit
from org.orekit.forces import ForceModel, BoxAndSolarArraySpacecraft, Panel
from org.orekit.propagation import SpacecraftState
from org.orekit.utils import Constants

import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone

def TEME_to_EME2000(teme_pos, teme_vel, mjds):
    # Orekit Frames
    frame_TEME = FramesFactory.getTEME()
    frame_EME2000 = FramesFactory.getEME2000()

    # Prepare output arrays
    eme2000_pos = np.empty_like(teme_pos)
    eme2000_vel = np.empty_like(teme_vel)

    # Iterate over each row of position, velocity, and corresponding MJD
    for i in range(len(teme_pos)):
        # Convert MJD to Julian Date and then to UTC datetime
        mjd = mjds.iloc[i]
        jd = mjd + 2400000.5
        base_date = datetime(1858, 11, 17, tzinfo=timezone.utc)
        
        # Convert to datetime
        dt = base_date + timedelta(days=(jd - 2400000.5))
        
        # Convert datetime to AbsoluteDate
        absolute_date = datetime_to_absolutedate(dt)

        # Convert inputs to Orekit's Vector3D and PVCoordinates (and convert from km to m)
        teme_pos_vector = Vector3D(float(teme_pos[i, 0] * 1000), float(teme_pos[i, 1] * 1000), float(teme_pos[i, 2] * 1000))
        teme_vel_vector = Vector3D(float(teme_vel[i, 0] * 1000), float(teme_vel[i, 1] * 1000), float(teme_vel[i, 2] * 1000))
        pv_teme = PVCoordinates(teme_pos_vector, teme_vel_vector)

        # Transform Coordinates from TEME to EME2000
        teme_to_eme2000 = frame_TEME.getTransformTo(frame_EME2000, absolute_date)
        pv_eme2000 = teme_to_eme2000.transformPVCoordinates(pv_teme)

        # Extract position and velocity from transformed coordinates and convert back from m to km
        eme2000_pos[i] = np.array([pv_eme2000.getPosition().getX(), pv_eme2000.getPosition().getY(), pv_eme2000.getPosition().getZ()]) / 1000
        eme2000_vel[i] = np.array([pv_eme2000.getVelocity().getX(), pv_eme2000.getVelocity().getY(), pv_eme2000.getVelocity().getZ()]) / 1000

    return eme2000_pos, eme2000_vel

import datetime 
from sgp4.api import Satrec
from typing import Dict, List, Union, Tuple, Any 

def sgp4_prop_TLE(TLE: str, jd_start: float, jd_end: float, dt: float, alt_series: bool = False) -> List[List[Any]]:
    """
    Given a TLE, a start time, end time, and time step, propagate the TLE and return the time-series of Cartesian coordinates and accompanying time-stamps (MJD).
    
    This is simply a wrapper for the SGP4 routine in the sgp4.api package (Brandon Rhodes).

    Parameters
    ----------
    TLE : str
        TLE to be propagated.
    jd_start : float
        Start time of propagation in Julian Date format.
    jd_end : float
        End time of propagation in Julian Date format.
    dt : float
        Time step of propagation in seconds.
    alt_series : bool, optional
        If True, return the altitude series as well as the position series. Defaults to False.

    Returns
    -------
    list
        List of lists containing the time-series of Cartesian coordinates, and accompanying time-stamps (MJD).
    """
    if jd_start > jd_end:
        print('jd_start must be less than jd_end')
        return

    ephemeris = []
    
    #convert dt from seconds to julian day
    dt_jd = dt/86400

    #split at the new line
    split_tle = TLE.split('\n')
    s = split_tle[0]
    r = split_tle[1]

    fr = 0.0 # precise fraction (SGP4 docs for more info)
    
    #create a satellite object
    satellite = Satrec.twoline2rv(s, r)

    time = jd_start
    # for i in range (jd_start, jd_end, dt):
    while time < jd_end:
        # propagate the satellite to the next time step
        # Position is in idiosyncratic True Equator Mean Equinox coordinate frame used by SGP4
        # Velocity is the rate at which the position is changing, expressed in kilometers per second
        error, position, velocity = satellite.sgp4(time, fr)
        if error != 0:
            print('Satellite position could not be computed for the given date')
            break
        else:
            ephemeris.append([time,position, velocity]) #jd time, pos, vel
        time += dt_jd

    return ephemeris

def TLE_time(TLE: str) -> float:
    """
    Find the time of a TLE in Julian Day format.

    Parameters
    ----------
    TLE : str
        The TLE string.

    Returns
    -------
    float
        Time in Julian Day format.
    """
    #find the epoch section of the TLE
    epoch = TLE[18:32]
    #convert the first two digits of the epoch to the year
    year = 2000+int(epoch[0:2])
    
    # the rest of the digits are the day of the year and fractional portion of the day
    day = float(epoch[2:])
    #convert the day of the year to a day, month, year format
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(day - 1)
    #convert the date to a julian date
    jd = (date - datetime.datetime(1858, 11, 17)).total_seconds() / 86400.0 + 2400000.5
    return jd

def combine_TLE2eph(TLE_list: List[str], jd_start: float, jd_stop: float, dt: float = 60) -> Tuple[List[Any], List[Any]]:
    dt_jd = dt / 86400
    current_jd = jd_start
    ephemeris = []
    orbit_ages = []

    # Iterate through the TLE list
    for i in range(len(TLE_list)):
        TLE_jd = TLE_time(TLE_list[i])
        # If it's the last TLE, we can't fetch the next TLE's JD straightforwardly
        if i < len(TLE_list) - 1:
            next_TLE_jd = TLE_time(TLE_list[i + 1])
        else:
            # If there's no next TLE, use jd_stop as the next TLE's JD for the last iteration
            next_TLE_jd = jd_stop

        # Process ephemeris only if the current JD is within the range of the current TLE's JD and the next one
        while current_jd < next_TLE_jd and current_jd < jd_stop:
            eph = sgp4_prop_TLE(TLE_list[i], current_jd, min(current_jd + dt_jd, jd_stop), dt)
            ephemeris.extend(eph)
            hours_orbit_age = (current_jd - TLE_jd) * 24
            orbit_ages.append(hours_orbit_age)
            current_jd += dt_jd
            if current_jd >= jd_stop:
                break

    return ephemeris, orbit_ages

from astropy.time import Time
def utc_to_jd(utc_time: datetime) -> float:
    """
    Convert UTC time (datetime object) to Julian Date using Astropy,
    rounding to the nearest full second to avoid timing errors.

    Parameters
    ----------
    utc_time : datetime
        UTC time tag.

    Returns
    -------
    float
        Julian Date.
    """
    # Round the input datetime to the nearest full second
    if utc_time.microsecond >= 500000:
        rounded_utc_time = utc_time + datetime.timedelta(seconds=1)
        rounded_utc_time = rounded_utc_time.replace(microsecond=0)
    else:
        rounded_utc_time = utc_time.replace(microsecond=0)

    # Convert the rounded datetime object to Astropy Time object
    time = Time(rounded_utc_time, format='datetime', scale='utc', precision=9)

    # Convert to Julian Date
    jd = time.jd
    return jd


Downloaded SP3 file for 2022-12-31 00:00:00.000 in sp3_download
Downloaded SP3 file for 2023-01-01 00:00:00.000 in sp3_download
Downloaded SP3 file for 2023-01-02 00:00:00.000 in sp3_download


In [7]:
#move back to the root directory
import os
os.chdir('..')

In [153]:
!pwd

/Users/charlesc/Documents/GitHub/ERP_tools


In [157]:
from source.tools.sp3_2_ephemeris import sp3_ephem_to_df

In [10]:
navstar_46_ephem = sp3_ephem_to_df('NAVSTAR46')

In [11]:
navstar_46_ephem

,UTC,x,y,z,xv,yv,zv,sigma_x,sigma_y,sigma_z,sigma_xv,sigma_yv,sigma_zv
0,2022-12-30 23:59:42,-2.125476e+07,1.602083e+07,-1.031196e+06,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001
1,2022-12-31 00:04:42,-2.167465e+07,1.550874e+07,-8.064242e+04,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001
2,2022-12-31 00:09:42,-2.205347e+07,1.496726e+07,8.700645e+05,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001
3,2022-12-31 00:14:42,-2.239057e+07,1.439747e+07,1.819126e+06,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001
4,2022-12-31 00:19:42,-2.268540e+07,1.380050e+07,2.764752e+06,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,2023-01-02 23:39:42,-2.047013e+07,1.681358e+07,-2.535468e+06,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001
861,2023-01-02 23:44:42,-2.095469e+07,1.635126e+07,-1.588851e+06,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001
862,2023-01-02 23:49:42,-2.139944e+07,1.585786e+07,-6.392138e+05,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001
863,2023-01-02 23:54:42,-2.180359e+07,1.533438e+07,3.116369e+05,NaN,NaN,NaN,0.5,0.5,0.5,0.001,0.001,0.001


In [129]:
from datetime import datetime, timedelta
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import getpass

startdate = datetime(2022, 12, 28, 0, 00, 00, 00000)
# Extending collection duration to test data availability
duration_days = 8  # days
enddate = startdate + timedelta(days=duration_days)

identity_st = input('Enter SpaceTrack username: ')
password_st = getpass.getpass(prompt=f'Enter SpaceTrack password for account {identity_st}: ')

st = SpaceTrackClient(identity=identity_st, password=password_st)

try:
    rawTles = st.tle(norad_cat_id=25933, epoch=op.inclusive_range(startdate, enddate),
                     orderby='epoch asc', format='tle')
    if rawTles:
        print("rawTles: ", rawTles)
    else:
        print("No TLE data found for the specified range.")
except Exception as e:
    print(f"An error occurred: {e}")



rawTles:  1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000+0 0  9996
2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195
1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000-0 0  9997
2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195
1 25933U 99055A   22365.15420913 -.00000095  00000-0  00000-0 0  9998
2 25933  52.3493 350.2064 0135623 136.9958  42.0236  2.00571908170222
1 25933U 99055A   22365.90803167 -.00000098  00000-0  00000-0 0  9990
2 25933  52.3491 350.1740 0135592 137.0076 226.3365  2.00571904170246
1 25933U 99055A   23001.90345068 -.00000101  00000-0  00000-0 0  9994
2 25933  52.3488 350.1313 0135551 137.0227 225.1000  2.00571904170266
1 25933U 99055A   23002.39538769 -.00000102  00000-0  00000-0 0  9991
2 25933  52.3486 350.1102 0135532 137.0311 220.3126  2.00571904170269
1 25933U 99055A   23002.89079448 -.00000103  00000-0  00000-0 0  9991
2 25933  52.3485 350.0889 0135512 137.0385 218.0315  2.00571902170271
1 25933U 9

In [134]:
from collections import OrderedDict

# Split the raw string by newlines
tle_lines = rawTles.strip().split('\n')

# Group every two lines to form individual TLEs
tle_list = ['\n'.join(tle_lines[i:i+2]) for i in range(0, len(tle_lines), 2)]

tle_list = list(OrderedDict.fromkeys(tle_list))

print(tle_list)

['1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000+0 0  9996\n2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195', '1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000-0 0  9997\n2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195', '1 25933U 99055A   22365.15420913 -.00000095  00000-0  00000-0 0  9998\n2 25933  52.3493 350.2064 0135623 136.9958  42.0236  2.00571908170222', '1 25933U 99055A   22365.90803167 -.00000098  00000-0  00000-0 0  9990\n2 25933  52.3491 350.1740 0135592 137.0076 226.3365  2.00571904170246', '1 25933U 99055A   23001.90345068 -.00000101  00000-0  00000-0 0  9994\n2 25933  52.3488 350.1313 0135551 137.0227 225.1000  2.00571904170266', '1 25933U 99055A   23002.39538769 -.00000102  00000-0  00000-0 0  9991\n2 25933  52.3486 350.1102 0135532 137.0311 220.3126  2.00571904170269', '1 25933U 99055A   23002.89079448 -.00000103  00000-0  00000-0 0  9991\n2 25933  52.3485 350.0889 0135512 137.0385 218.0315  2.00571902

In [148]:
jd_start = utc_to_jd(datetime.datetime(2022, 12, 29, 0, 0, 0))
jd_end = utc_to_jd(datetime.datetime(2023, 1, 3, 0, 0, 0))
navstar46TLE_eph = combine_TLE2eph(tle_list, jd_start, jd_end, dt=60)


2459942.5
2459947.5
split_tle: ['1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000+0 0  9996', '2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195']
split_tle: ['1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000+0 0  9996', '2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195']
split_tle: ['1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000+0 0  9996', '2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195']
split_tle: ['1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000+0 0  9996', '2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195']
split_tle: ['1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000+0 0  9996', '2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195']
split_tle: ['1 25933U 99055A   22363.89832389 -.00000091  00000-0  00000+0 0  9996', '2 25933  52.3497 350.2603 0135678 136.9754 215.1860  2.00571911170195']
split_tle: ['1 25933U 99055A   2

In [167]:
import numpy as np
import pandas as pd

# Prepare the input arrays for the TEME_to_EME2000 function
teme_pos = np.array([item[1] for item in navstar46TLE_eph[0]])
teme_vel = np.array([item[2] for item in navstar46TLE_eph[0]])
mjds = pd.Series([item[0] for item in navstar46TLE_eph[0]])

# Call the TEME_to_EME2000 function
eme2000_pos, eme2000_vel = TEME_to_EME2000(teme_pos, teme_vel, mjds)

NameError: name 'navstar46TLE_eph' is not defined